In [3]:
import pandas as pd

import tabula

In [4]:
table = tabula.read_pdf('papers/haunschild2012.pdf',pages=4)

df = table[0]

df.columns= df.loc[0]


part1 = df.iloc[1:,[0,1,6]]
part2 = df.iloc[1:,[10,11,16]]

part1.columns = ['Formula','Name','E']
part2.columns = ['Formula','Name','E']

df1 = pd.concat([part1,part2]).reset_index(drop=True)

Got stderr: Nov 25, 2020 1:27:18 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 25, 2020 1:27:18 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [5]:
table = tabula.read_pdf('papers/haunschild2012.pdf',pages=5)

df = table[0]

df.columns= df.loc[0]

df = df.iloc[1:,[0,1,6]]

df2 = df.dropna().iloc[:-4,:]

Got stderr: Nov 25, 2020 1:27:19 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Nov 25, 2020 1:27:20 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 25, 2020 1:27:20 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [6]:
df2.columns = ['Formula','Name','E']
df = pd.concat([df1.iloc[1:],df2])

In [7]:
df = df.reset_index(drop=True)

In [8]:
df['Formula'] = df['Formula'].apply(lambda x: x.replace(' ',''))

In [9]:
df[~df['Formula'].apply(lambda x: len(x) < 10)]

,Formula,Name,E
130,"valuesfromRef.43,theW4valuesfromRef.9,andtheG2...",Dioxygen,505.88
142,leveloftheoryis0.47kJ/mollowerthanforthestruct...,Triplet silylene,555.72
144,SiHmulti-referenceeffectsinsingletCH2arenotsuf...,Silane,1357.91


In [10]:
df.loc[130,'Formula'] = 'O2'
df.loc[142,'Formula'] = 'SiH2'
df.loc[144,'Formula'] = 'SiH4'
df.loc[132,'Formula'] = 'O3'

In [11]:
from itertools import permutations
from ase.io import read, write
import numpy as np 
from ase import Atoms

In [12]:
def expand_molname(name):
    import re
    def it(l):
        if not isinstance(l,list):
            l = [l]
        for l_ in l:
            yield l_ 
    
    # Splitting on UpperCase using re
    res_list = []
    res_list = re.findall('[A-Z][^A-Z]*', name)
    for i, a in enumerate(res_list):
        if a[-1].isnumeric():
            res_list[i] = [a[:-1]]*int(a[-1])
    res_list = [r for l in res_list for r in it(l)]
    return res_list

In [32]:
df['xyz_idx']= [[] for _ in range(len(df))]

In [33]:
for a_idx, a in enumerate(read('g2_97.xyz',':')):
    for loc, itm in df.iterrows():
        form = itm.Formula
        try:
            df_num = np.sort(Atoms(form).get_atomic_numbers())
        except ValueError:
            print(loc, form)
            raise ValueError
        a_num = np.sort(a.get_atomic_numbers())
        if len(df_num) == len(a_num) and np.allclose(df_num, a_num):
            df.loc[loc, 'xyz_idx'].append(a_idx)

In [42]:
df.to_csv('g2_97.csv')

In [57]:
df = pd.read_csv('g2_97.csv',index_col=0)

In [58]:
df['xyz_idx'] = df.xyz_idx.apply(lambda x: int(x.replace('[','').replace(']','')))

In [61]:
df = df.set_index('xyz_idx')

In [63]:
df.to_csv('g2_97.csv')

In [64]:
df

,Formula,Name,E
xyz_idx,,,
67,AlCl3,Aluminum trichloride,1310.70
41,AlF3,Aluminum trifluoride,1807.99
61,BCl3,Boron trichloride,1359.87
133,BF3,Boron trifluoride,1972.30
14,BeH,Beryllium monohydride,212.50
...,...,...,...
49,SiH3,Silyl radical,953.44
50,SiH4,Silane,1357.91
31,SiO,Silicon monoxide,809.19


In [70]:
from ase.calculators.singlepoint import SinglePointCalculator

In [71]:
from ase.units import kJ,mol
kJpmol = kJ/mol

In [72]:
atoms = []
for a_idx, a in enumerate(read('g2_97.xyz',':')):
    a.calc = SinglePointCalculator(a)
    a.calc.results['energy'] = df.loc[a_idx,'E'] * kJpmol
    a.info['name'] = df.loc[a_idx,'Name']
    atoms.append(a)

In [74]:
write('g2_97.traj', atoms)

In [76]:
import PyPDF2 as pdf

# importing required modules 
import PyPDF2 
  
# creating a pdf file object 
pdfFileObj = open('papers/haunschildsupplementalmaterial.pdf', 'rb') 
  
# creating a pdf reader object 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
  
# printing number of pages in pdf file 
print(pdfReader.numPages) 
  
text = ''
# creating a page object 
for p in range(2,45):
    pageObj = pdfReader.getPage(p) 
  
    # extracting text from page 
    page = pageObj.extractText()
    text += page
# closing the pdf file object 
pdfFileObj.close() 


lines = text.split()


coords = []
new_mol = True
cnt = 2
name = ''
for i,l in enumerate(lines):
    if len(l) < 3 and l.lower() == l and not l.isnumeric():
        if cnt > 1:
            name = lines[i-2]
        coords.append([l,name, lines[i-1]])
        cnt = 0
    else:
        cnt += 1

def coords_from_str(coords):
    try:
        
        c = coords.replace(' ','')
        c = c.split('.')
        x = c[0] +'.'+ c[1][:14]
        y = c[1][14:] +'.'+ c[2][:14]
        z = c[2][14:] +'.'+ c[3]
        x,y,z = float(x),float(y),float(z)
        return x,y,z
    except Exception:
        return coords
    
coords = [[c[0].upper(),c[1],coords_from_str(c[2])] for c in coords]

In [236]:
text = ''
mol = ''
cont = [] 
def flush(cont):
    t = ''
    try:
        if cont:
            t = """{} \n {} \n""".format(len(cont),cont[0][1])
            for c in cont:
                t += "{}\t{}\t{}\t{} \n".format(c[0], *c[2])
    except Exception:
        pass
    return t

for c in coords:
    if c[1] != mol:
        mol = c[1]
        text += flush(cont)
        cont = []
    cont.append(c) 

In [238]:
with open('haunshild_coords.xyz','w') as file:
    file.write(text)